<a href="https://colab.research.google.com/github/ytg000629/daegu_bigdata/blob/main/TEXTRANK%EB%AC%B8%EC%9E%A5%EC%9A%94%EC%95%BD_TTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install emoji
!pip install konlpy
!pip install google-cloud-texttospeech
!pip install google-cloud-speech
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
dpath = "/content/drive/MyDrive/대구 빅데이터/data/"

In [10]:
import pandas as pd
import numpy as np
import emoji
import re
from tqdm import tqdm
from nltk.corpus import stopwords
import nltk
import string
import networkx
import re
from konlpy.tag import Komoran
from google.cloud import texttospeech
from google.cloud import speech
import os
import io
import warnings

warnings.filterwarnings("ignore")


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
data = pd.read_csv(dpath+'대구 소비.csv')

In [34]:
data.head(20)

,date,title,content
0,2023-06-22 14:27:02,대구 경제지표 역대 최고치…전년 대비 소비자 물가 3.7% 상승,[\n경제성장률·고용·수출·투자유치 등 역대 최고치전기·가스·수도 요금 등 전년 대...
1,2023-06-23 13:55:07,"대구시, 소금 구매 허위·과장광고 주의보 발령",[\n일본 후쿠시마 원전 오염수 방류와 소금 생산량 감소 영향\n\n\n\n[대구=...
2,2023-06-23 14:08:08,"[이앤피] ""홍준표 선거법 위반 고발사건, 경찰 대구시청 압수수색"" 外",[\n\n\n\n\n■ 방송 : YTN 라디오 FM 94.5 (13:00~14:00...
3,2023-06-22 15:54:01,"개그맨 김민경씨, 대구시에 고향사랑기부금 500만원 기탁","[\n\t\t\t대구시는 예능, 스포츠, 광고 등 다양한 활동을 펼치고 있는 대구 ..."
4,2023-06-19 16:51:09,"[대구소식] 농협대구본부, 군위 대구편입 맞춰 현지 농협 지원",[\n\n\n\n\n군위군 산지농협 지원 확대[농협대구본부 제공. 재판매 및 DB ...
5,2023-06-22 17:18:01,"디자인진흥원-대구시, 산업부 지역 퍼스널케어 산업 고도화 지원 나서","[\n오는 7월 12일까지 참여기업 모집, 제품디자인, 브랜드, 콘테츠 등 분야별 ..."
6,2023-06-22 15:11:04,"평년보다 긴 장마 예상에…대구 신세계, 레인부츠 상품 강화","[\n\n\n\n\n[대구=뉴시스] 대구 신세계백화점은 바버, 핏플랍 등 레인부츠 ..."
7,2023-06-22 14:47:01,"권기훈 대구시의원, '1회용품 사용 줄이기 조례' 제정안 발의","[\n1회용품 사용 줄이기 위한 제도적 장치 마련기후변화와 환경오염에 적극 대응, ..."
8,2023-06-22 14:48:01,"한국참외생산자협의회, 참외 소비촉진 나눔행사 개최",[\n\n\n\n\n한국참외생산자협의회·참외자조금관리위원회와 농협경제지주는 21일 ...
9,2023-06-21 16:10:00,"한전KDN, 대구 ‘2023 전기산업엑스포’ 참가",[\n \n\n\n\n 한전KDN이 지난 14부터 16일까지 대...


In [35]:
for i in range(len(data)):
    #data.content[i] = data.content[i].replace('\n','')
    #data.content[i] = data.content[i].replace('\t','')
    data.content[i] = data.content[i].replace('■','')
    data.content[i] = data.content[i].replace('▲','')

In [36]:
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')

    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')

    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0


    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1

        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1

        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])

    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))

        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)

        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s

    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])

    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])

    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))

    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')

    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi

        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)

        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)

        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)

        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)

        return both

    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

In [37]:
result = []
for i in range(len(data)):
    f = open(f"content.txt{i}", 'w')
    f.write(data.content[i])
    f.close()
    tr = TextRank()
    tagger = Komoran()
    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
    tr.loadSents(RawSentenceReader(f'content.txt{i}'), lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
    tr.build()
    ranks = tr.rank()
    #for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    #print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
    #print("\t".join([str(tr.dictCount[k])]))
    print(f'{i}.' , tr.summarize(0.2))
    a= tr.summarize(0.2)
    result.append(a)
    print('')

0.  / 대구 = 박성원 기자22일 기준 대구 소비자 물가지수가 전년 동월 대비 3.7%가 올라 서민들의 가정 경제 부담이 커진 가운데 대구 경제지표가 역대 최고치를 기록했다. 대구시는 22일 오전 동인청사에서 ‘민선8기 1주년 대구 경제현황’ 기자회견을 통해 대구 경제성장률, 고용, 수출, 투자유치 등 주요 경제지표가 역대 최고치를 기록했다고 밝혔다.2023년 1분기 대구시 경제성장률은 전년 같은 기간 대비 3.8% 상승해 전국 경제성장률 0.9%를 크게 웃도는 상승세를 보이고 있다.  실업률도 2.9%로 전년 같은 기간 대비 0.5% 감소했다.

1. 현재 전국 곳곳에서는 ‘소금사재기’ 열풍이 일면서 천일염 품절사태가 이어지고 있다. 상담 내용을 상세 분석한 결과, 일본 원전 오염수 방류에 따른 불안감과 불확실한 정보로 소비자의 구매를 부추기는 판매자의 허위·과장광고로 인해 소금 구매가 급증했다.

2. [이앤피] "홍준표 선거법 위반 고발사건, 경찰 대구시청 압수수색" 外 ◇ 이승훈 앵커(이하 이승훈) :  영특하고 민첩한 뉴스캐스터, 김영민 아나운서와 함께합니다.  경찰이 대구시청에 대한 압수수색에 나섰죠?  이 대목에서 이번 압수수색을 지난번 대구 퀴어문화축제에서 경찰과 대구시 측이 충돌한 것에 대한 보복으로 규정했음을 알 수 있습니다.  그러면서 "수사권을 그런 식으로 행사하면 경찰이 아니라 그건 깡패다.  경찰은 어떤 입장인가요?  경찰 관계자는 홍 시장의 입장과 관련해 "지난 9일 검찰에서 압수수색 영장을 신청했고, 지난 16일 법원에서 영장이 발부됐다.  타이태닉호 관광용 잠수정 '타이탄'의 탑승객 전원이 결국 사망했네요.  잠수정 운영사인 오션게이트도 성명을 내고 이들의 사망 사실을 확인했습니다.  유족은 A씨가 사망한 경위를 두고 CCTV를 확인하고 싶다는 뜻을 전했는데요.  이에 대사관은 현지 경찰을 통해 16일 법의학연구소에서 A씨로 추정되는 시신을 발견했고요.  A씨는 결혼을 앞두고 상견례까지 마친 상태였다고 합니다.  이에 경찰은 

In [17]:
result = []
for i in range(len(data)):
    f = open(f"content.txt{i}", 'w')
    f.write(data.content[i])
    f.close()
    tr = TextRank()
    tagger = Komoran()
    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
    tr.loadSents(RawSentenceReader(f'content.txt{i}'), lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
    tr.build()
    ranks = tr.rank()
    #for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    #print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
    #print("\t".join([str(tr.dictCount[k])]))
    print(f'{i}.' , tr.summarize(0.2))
    a= tr.summarize(0.2)
    result.append(a)
    print('')

0.  / 대구 = 박성원 기자22일 기준 대구 소비자 물가지수가 전년 동월 대비 3.7%가 올라 서민들의 가정 경제 부담이 커진 가운데 대구 경제지표가 역대 최고치를 기록했다. 대구시는 22일 오전 동인청사에서 ‘민선8기 1주년 대구 경제현황’ 기자회견을 통해 대구 경제성장률, 고용, 수출, 투자유치 등 주요 경제지표가 역대 최고치를 기록했다고 밝혔다.2023년 1분기 대구시 경제성장률은 전년 같은 기간 대비 3.8% 상승해 전국 경제성장률 0.9%를 크게 웃도는 상승세를 보이고 있다.  실업률도 2.9%로 전년 같은 기간 대비 0.5% 감소했다.

1. 현재 전국 곳곳에서는 ‘소금사재기’ 열풍이 일면서 천일염 품절사태가 이어지고 있다. 상담 내용을 상세 분석한 결과, 일본 원전 오염수 방류에 따른 불안감과 불확실한 정보로 소비자의 구매를 부추기는 판매자의 허위·과장광고로 인해 소금 구매가 급증했다.

2. [이앤피] "홍준표 선거법 위반 고발사건, 경찰 대구시청 압수수색" 外 ◇ 이승훈 앵커(이하 이승훈) :  영특하고 민첩한 뉴스캐스터, 김영민 아나운서와 함께합니다.  경찰이 대구시청에 대한 압수수색에 나섰죠?  이 대목에서 이번 압수수색을 지난번 대구 퀴어문화축제에서 경찰과 대구시 측이 충돌한 것에 대한 보복으로 규정했음을 알 수 있습니다.  그러면서 "수사권을 그런 식으로 행사하면 경찰이 아니라 그건 깡패다.  경찰은 어떤 입장인가요?  경찰 관계자는 홍 시장의 입장과 관련해 "지난 9일 검찰에서 압수수색 영장을 신청했고, 지난 16일 법원에서 영장이 발부됐다.  타이태닉호 관광용 잠수정 '타이탄'의 탑승객 전원이 결국 사망했네요.  잠수정 운영사인 오션게이트도 성명을 내고 이들의 사망 사실을 확인했습니다.  유족은 A씨가 사망한 경위를 두고 CCTV를 확인하고 싶다는 뜻을 전했는데요.  이에 대사관은 현지 경찰을 통해 16일 법의학연구소에서 A씨로 추정되는 시신을 발견했고요.  A씨는 결혼을 앞두고 상견례까지 마친 상태였다고 합니다.  이에 경찰은 

In [38]:
#Google Cloud Text To Speech(TTS) API
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/drive/MyDrive/대구 빅데이터/daegubigdata-d6b45b2dabc8.json"

def TextToSpeech(text, file_name):
    client = texttospeech.TextToSpeechClient()
    input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(language_code = 'ko-KR', ssml_gender = texttospeech.SsmlVoiceGender.FEMALE)
    audio_config = texttospeech.AudioConfig(audio_encoding = texttospeech.AudioEncoding.LINEAR16)
    response = client.synthesize_speech(input = input, voice = voice, audio_config = audio_config)

    with open(file_name, 'wb') as out:
        out.write(response.audio_content)

In [40]:
#Colab 오디오를 통해 듣기
from IPython.display import Audio

TextToSpeech('外', "./Answer.wav")
audio_file = '/content/Answer.wav'
Audio(audio_file)